In [ ]:
import mlflow
import os

# os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-54-237-230-195.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://ec2-54-237-230-195.compute-1.amazonaws.com:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-22143/1', creation_time=1746582046600, experiment_id='1', last_update_time=1746582046600, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-22143/0', creation_time=1746579543436, experiment_id='0', last_update_time=1746579543436, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
import boto3, json
s3 = boto3.client("s3")
print(s3.list_buckets()["Buckets"])

[{'Name': 'cdk-hnb659fds-assets-852998998504-us-east-1', 'CreationDate': datetime.datetime(2025, 5, 3, 3, 5, 43, tzinfo=tzlocal())}, {'Name': 'mlflow-artifacts-remote-22143', 'CreationDate': datetime.datetime(2025, 5, 7, 0, 32, 52, tzinfo=tzlocal())}]


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/07 02:24:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifacts-remote-22143/1/051515c6acb9414ba689d8c79568bb70/artifacts'
🏃 View run skittish-quail-25 at: http://ec2-54-237-230-195.compute-1.amazonaws.com:5000/#/experiments/1/runs/051515c6acb9414ba689d8c79568bb70
🧪 View experiment at: http://ec2-54-237-230-195.compute-1.amazonaws.com:5000/#/experiments/1


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-22143/1', creation_time=1746582046600, experiment_id='1', last_update_time=1746582046600, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-22143/0', creation_time=1746579543436, experiment_id='0', last_update_time=1746579543436, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [11]:
run_id =  client.search_runs(experiment_ids='1')[0].info.run_id 
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)


Successfully registered model 'iris-classifier'.
2025/05/07 02:24:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1746584668571, current_stage='None', description='', last_updated_timestamp=1746584668571, name='iris-classifier', run_id='051515c6acb9414ba689d8c79568bb70', run_link='', source='s3://mlflow-artifacts-remote-22143/1/051515c6acb9414ba689d8c79568bb70/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>